# YAML-Based Workflow Example

This notebook demonstrates how to define and run Magentic workflows using YAML configuration files.

In [ ]:
import os
from pathlib import Path

# Ensure OPENAI_API_KEY is set
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

print("✓ Environment ready")

## Step 1: Load Workflow from YAML

The workflow configuration is defined in `config/workflows/magentic_example.yaml`.
This YAML file specifies:
- Orchestrator settings (max rounds, stalls, resets)
- Manager/planner configuration (model, instructions)
- Agent definitions (researcher, coder with their models and tools)
- Callback settings for observability

In [ ]:
from agenticfleet.fleet.workflow_loader import load_workflow_from_yaml

# Load the workflow configuration
config_path = Path("../var/config/workflows/magentic_example.yaml")
workflow_config = load_workflow_from_yaml(config_path)

print(f"Workflow: {workflow_config['name']}")
print(f"Agents: {[a['name'] for a in workflow_config['agents']]}")
print("✓ Configuration loaded")

## Step 2: Build Workflow from Configuration

Create the actual Magentic workflow using the YAML configuration.

In [ ]:
from agenticfleet.fleet.callbacks import ConsoleCallbacks
from agenticfleet.fleet.workflow_loader import build_workflow_from_config

# Create console callbacks for observability
callbacks = ConsoleCallbacks()

# Build the workflow
workflow = build_workflow_from_config(workflow_config, console_callbacks=callbacks)

print("✓ Workflow built successfully")

## Step 3: Define Task

Specify the research and analysis task for the workflow.

In [ ]:
task = (
    "I am preparing a report on renewable energy trends. "
    "Research the latest developments in solar and wind energy efficiency (2024-2025). "
    "Then calculate the projected cost reduction percentages if efficiency improves by 15% "
    "for solar and 20% for wind, assuming current costs of $0.05/kWh "
    "for solar and $0.04/kWh for wind. Provide a summary with tables "
    "showing current vs. projected costs."
)

print(f"Task: {task[:100]}...")

## Step 4: Run Workflow

Execute the workflow with streaming events.

In [ ]:
from agent_framework import WorkflowOutputEvent


async def run_yaml_workflow():
    print("\n" + "=" * 50)
    print("Starting YAML-configured workflow...")
    print("=" * 50 + "\n")

    try:
        output = None
        async for event in workflow.run_stream(task):
            if isinstance(event, WorkflowOutputEvent):
                output = str(event.data)

        if output:
            print(f"\n\nWorkflow Result:\n{output}")
        else:
            print("\n\nWorkflow completed (no output)")
    except Exception as e:
        print(f"\n\nWorkflow failed: {e}")
        raise


# Execute the workflow
await run_yaml_workflow()

## Alternative: One-Step Loading

You can also load and build the workflow in a single call:

In [ ]:
from agenticfleet.fleet.workflow_loader import load_and_build_workflow

# Load and build in one step
workflow_simple = load_and_build_workflow("../config/workflows/magentic_example.yaml")

print("✓ Workflow loaded and built in one step")
print("\nYou can now run: await workflow_simple.run_stream(task)")